In [138]:
import secrets
import time

from py_ecc.optimized_bls12_381 import G1, G2, curve_order,add, multiply, pairing
#from mpyc.runtime import mpc

In [139]:
class BBSPlus:
    def __init__(self, ell: int):
        self.ell = ell
        self.g1 = G1
        self.g2 = G2
        self.H = [multiply(G1, secrets.randbelow(curve_order)) for _ in range(ell + 1)]
        self.x = secrets.randbelow(curve_order)
        self.X = multiply(G2, self.x)

    def sign(self, messages: list[int]) -> tuple:
        if len(messages) != self.ell:
            raise ValueError(f"Expected {self.ell} messages, got {len(messages)}")
        e = secrets.randbelow(curve_order)
        s = secrets.randbelow(curve_order)
        B = self.g1
        B = add(B, multiply(self.H[0], s))
        for mi, Hi in zip(messages, self.H[1:]):
            mi_mod = mi % curve_order
            B = add(B, multiply(Hi, mi_mod))
        denom = (self.x + e) % curve_order
        inv = pow(denom, -1, curve_order)
        A = multiply(B, inv)
        return (A, e, s)

    def verify(self, messages: list[int], signature: tuple) -> bool:
        A, e, s = signature
        if len(messages) != self.ell:
            return False
        B = self.g1
        B = add(B, multiply(self.H[0], s))
        for mi, Hi in zip(messages, self.H[1:]):
            mi_mod = mi % curve_order
            B = add(B, multiply(Hi, mi_mod))
        left_input = add(self.X, multiply(self.g2, e % curve_order))  # G2 element
        left = pairing(left_input, A)
        right = pairing(self.g2, B)
        return left == right

In [140]:
def lagrange(j, J, q=curve_order):
    num = 1
    den = 1
    for m in J:
        if m == j:
            continue
        num = (num * (-m)) % q
        den = (den * (j - m)) % q
    inv_den = pow(den, -1, q)
    return (num * inv_den) % q

In [141]:
def poly_eval(coeffs, x, q=curve_order):
    res = 0
    power = 1
    for c in coeffs:
        res = (res + c * power) % q
        power = (power * x) % q
    return res

In [142]:
class ThresholdBBS:
    def __init__(self, ell: int, n: int, t: int):
        if not (1 <= t <= n):
            raise ValueError("Threshold t must satisfy 1 <= t <= n")
        self.ell = ell
        self.n = n
        self.t = t
        # Use BBSPlus to generate public params and master secret x0
        base = BBSPlus(ell)
        self.g1 = base.g1
        self.g2 = base.g2
        self.H = base.H
        self.X = base.X
        # Master secret to be shared
        x0 = base.x
        self._poly = [x0] + [secrets.randbelow(curve_order) for _ in range(t-1)]
        self.shares = [ poly_eval(self._poly, i+1) for i in range(n) ]

    def threshold_sign(self, messages: list[int], subset: list[int]) -> tuple:
        if len(subset) != self.t:
            raise ValueError(f"Need exactly {self.t} shares to sign")
        if any(i < 0 or i >= self.n for i in subset):
            raise ValueError("Subset indices out of range")
        J = [i+1 for i in subset]
        x_recon = 0
        for j in J:
            lam = lagrange(j, J)
            xj = self.shares[j-1]
            x_recon = (x_recon + lam * xj) % curve_order
        # Use standard BBS+ sign algorithm with x_recon
        e = secrets.randbelow(curve_order)
        s = secrets.randbelow(curve_order)
        # Compute B
        B = self.g1
        B = add(B, multiply(self.H[0], s))
        for mi, Hi in zip(messages, self.H[1:]):
            mi_mod = mi % curve_order
            B = add(B, multiply(Hi, mi_mod))
        # Compute A = B^(1/(x_recon + e))
        denom = (x_recon + e) % curve_order
        inv = pow(denom, -1, curve_order)
        A = multiply(B, inv)
        return (A, e, s)



    def verify(self, messages: list[int], signature: tuple) -> bool:
        A, e, s = signature
        if len(messages) != self.ell:
            return False
        B = self.g1
        B = add(B, multiply(self.H[0], s))
        for mi, Hi in zip(messages, self.H[1:]):
            mi_mod = mi % curve_order
            B = add(B, multiply(Hi, mi_mod))
        left = pairing(add(self.X, multiply(self.g2, e % curve_order)), A)
        right = pairing(self.g2, B)
        return left == right

In [ ]:
# =================== Benchmarking ===================
# Benchmarking measures the signing and verification speed, signature size, and correctness for both standard BBS+ and threshold BBS+ signature schemes.
# It compares how signature generation and verification behave under different parameters(e.g., number of messages, threshold settings, number of participants).

def benchmark_bbsplus(ell: int, repeat=3):
    messages = [42 + i for i in range(ell)]
    bbs = BBSPlus(ell)
    t0 = time.time()
    for _ in range(repeat):
        signature = bbs.sign(messages)
    t1 = time.time()
    t2 = time.time()
    for _ in range(repeat):
        valid = bbs.verify(messages, signature)
    t3 = time.time()
    sign_time = (t1 - t0) / repeat
    verify_time = (t3 - t2) / repeat
    sig_size = sum([len(str(x)) for x in signature])
    return {"scheme": "BBS+", "ell": ell, "sign_time": sign_time, "verify_time": verify_time, "sig_size": sig_size, "valid": valid}

def benchmark_thresholdbbs(ell, n, t, repeat=3):
    tbbs = ThresholdBBS(ell, n, t)
    messages = [42 + i for i in range(ell)]
    subset = list(range(t))
    t0 = time.time()
    for _ in range(repeat):
        signature = tbbs.threshold_sign(messages, subset)
    t1 = time.time()
    t2 = time.time()
    for _ in range(repeat):
        valid = tbbs.verify(messages, signature)
    t3 = time.time()
    sign_time = (t1 - t0) / repeat
    verify_time = (t3 - t2) / repeat
    sig_size = sum([len(str(x)) for x in signature])
    return {"scheme": "ThresholdBBS+", "ell": ell, "n": n, "t": t, "sign_time": sign_time, "verify_time": verify_time, "sig_size": sig_size, "valid": valid}

def run_benchmarks():
    print("==== BBS+ vs Threshold BBS+ Benchmark ====")
    print("---- BBS+ ----")
    for ell in [1, 3, 10, 30]:
        result = benchmark_bbsplus(ell)
        print(f"BBS+ ell={result['ell']}: sign:{result['sign_time']:.4f}s verify:{result['verify_time']:.4f}s size:{result['sig_size']} valid:{result['valid']}")
    print("---- Threshold BBS+ ----")
    for ell in [1, 3, 10]:
        for n, t in [(3,2), (5,3), (10,5)]:
            result = benchmark_thresholdbbs(ell, n, t)
            print(f"ThresholdBBS+ ell={result['ell']} n={result['n']} t={result['t']}: sign:{result['sign_time']:.4f}s verify:{result['verify_time']:.4f}s size:{result['sig_size']} valid:{result['valid']}")
    print("========================")

run_benchmarks()

==== BBS+ vs Threshold BBS+ Benchmark ====
---- BBS+ ----
BBS+ ell=1: sign:0.0135s verify:0.8124s size:504 valid:True
BBS+ ell=3: sign:0.0139s verify:0.8088s size:505 valid:True
BBS+ ell=10: sign:0.0147s verify:0.8015s size:504 valid:True
BBS+ ell=30: sign:0.0181s verify:0.7907s size:504 valid:True
---- Threshold BBS+ ----
ThresholdBBS+ ell=1 n=3 t=2: sign:0.0128s verify:0.7943s size:503 valid:True
ThresholdBBS+ ell=1 n=5 t=3: sign:0.0128s verify:0.7897s size:504 valid:True
ThresholdBBS+ ell=1 n=10 t=5: sign:0.0124s verify:0.7958s size:505 valid:True
ThresholdBBS+ ell=3 n=3 t=2: sign:0.0136s verify:0.7963s size:503 valid:True
ThresholdBBS+ ell=3 n=5 t=3: sign:0.0133s verify:0.8180s size:504 valid:True
ThresholdBBS+ ell=3 n=10 t=5: sign:0.0133s verify:0.8058s size:502 valid:True
ThresholdBBS+ ell=10 n=3 t=2: sign:0.0144s verify:0.7956s size:504 valid:True
ThresholdBBS+ ell=10 n=5 t=3: sign:0.0143s verify:0.9021s size:504 valid:True
ThresholdBBS+ ell=10 n=10 t=5: sign:0.0165s verify:0.95

In [ ]:
def string_to_int_list(str_list):
    return [int.from_bytes(s.encode('utf8'), 'big') % curve_order for s in str_list]


# verify that encoding real-world data preserves the security and correctness of the signature process.
print("==== BBS+ Real Data Test ====")
bbs_real = BBSPlus(3)
real_messages = ["yeshouxianbei", "114514", "z114514@ad.unsw.edu.au"]
messages_int = string_to_int_list(real_messages)
sig = bbs_real.sign(messages_int)
print("Messages (raw):", real_messages)
print("Messages (int):", messages_int)
print("Signature:", sig)
print("Verify:", bbs_real.verify(messages_int, sig))
print()

print("==== Threshold BBS+ Real Data Test ====")
tbbs_real = ThresholdBBS(3, 5, 3)

subset = [0, 2, 4]
sig2 = tbbs_real.threshold_sign(messages_int, subset)
print("Messages (raw):", real_messages)
print("Subset:", subset)
print("Signature:", sig2)
print("Verify:", tbbs_real.verify(messages_int, sig2))

==== BBS+ Real Data Test ====
Messages (raw): ['yeshouxianbei', '114514', 'z114514@ad.unsw.edu.au']
Messages (int): [9618005169869363825106802074985, 54087399059764, 45717513622168189320559085781022544088200214291046773]
Signature: ((1711442687937071435790327980301113381291924633484134135629951583082192732762370973878440403934176876653244581959675, 3631635760992362437741639730386613259109752195539427953794063684592986858286057857831733745503741079682246678312640, 871040181750037127693575950014128032406785878938886924292299706999611993697866043746027783764524574439818019555380), 3614552422778293047282573983264473056880162443584130319657311152719736844547, 33628787741273238285923422492735034840678497362487125796472509388069664770833)
Verify: True

==== Threshold BBS+ Real Data Test ====
Messages (raw): ['yeshouxianbei', '114514', 'z114514@ad.unsw.edu.au']
Subset: [0, 2, 4]
Signature: ((2094637501514506772121463924740655470729755721596233253861686479116615267551757215753211377868948938155